<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%20%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF_%EC%8B%A4%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- !pip install transformers -->

In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# core
import os, pymysql, shutil, unicodedata, pickle, re
import numpy as np
import pandas as pd

# visualize
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트
from matplotlib import rc # 폰트
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# tensorflow
import tensorflow as tf
## layers & models
from tensorflow.keras.layers import Layer, Input, Embedding, Concatenate, Flatten, Normalization
from tensorflow.keras.layers import Dense, LSTM, GRU, Attention
from tensorflow.keras.models import Sequential, Model
## optimizers & callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
## preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
## utils
from tensorflow.keras.utils import plot_model
## saving
from tensorflow.keras.saving import save_model
# Huggingface
from transformers import TFBertModel, FillMaskPipeline, TFRobertaModel, AutoModel, TFBertForSequenceClassification, TFGPT2LMHeadModel

from transformers import TFBertTokenizer, AutoTokenizer, AutoConfig
from transformers import logging
# scikit-learn
## models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

# Time Check
import time
from tqdm.notebook import tqdm
tqdm.pandas()

# customization
# from knockknock import desktop_sender
## font
# plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = True

os.environ["TOKENIZERS_PARALLELISM"] = "false"

logging.set_verbosity_error() #  fine-tuning할 때 warning 메시지 끄기

## preprocessed Data 불러오기

In [ ]:
df = pd.read_csv('/content/preprocessed_1.csv')
df.head()

,Unnamed: 0,ID,first_party,second_party,facts,first_party_winner,facts_split
0,0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1,"['On', 'June', '27,', '1962,', 'Phil', 'St.', ..."
1,1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0,"['Ramon', 'Nelson', 'was', 'riding', 'his', 'b..."
2,2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1,"['An', 'Alabama', 'state', 'court', 'convicted..."
3,3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0,"['Victor', 'Linkletter', 'was', 'convicted', '..."
4,4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1,"['On', 'April', '24,', '1953', 'in', 'Selma,',..."


# Configuration

In [ ]:
# 사전학습 모델 아카이브 (RoBERTa 아키텍쳐는 반영되지 않았습니다)
model_names = {0: 'nlpaueb/legal-bert-base-uncased', # BERT based
               1: 'saibo/legal-roberta-base', #RoBERTa based
               2: 'casehold/custom-legalbert', # BERT based
               3: 'nlpaueb/legal-bert-small-uncased' # BERT based
               }

# 사용할 모델 지정하기
MODEL_NAME = model_names[3]


# Hyper-Parameter
LEARNING_RATE = 1e-4

In [ ]:
class MyModel(tf.keras.Model): # Fine-Tune
    def __init__(self, model_name):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        self.classifier = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')

    def call(self, input_ids = None, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction


class WorkFlow(MyModel):
    def __init__(self, model_name):
        super().__init__(model_name)
        self.model_name = model_name
        self.config = AutoConfig.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)


    def call(self, input_ids=None, attention_mask=None, token_type_ids=None):
        super().call(input_ids = None, attention_mask=None, token_type_ids=None)



    def get_dataset(self, X_data, y_data, test_split=0.2, shuffle=True, shuffle_size=3000):

        tokens = self.tokenizer(X_data.tolist(), truncation=True, padding=True, return_tensors = 'tf')

        X_data = tokens
        y_data = y_data

        # dataset 타입으로 변환
        ds = tf.data.Dataset.from_tensor_slices((
            dict(X_data),
            y_data
        ))

        if shuffle:
            ds = ds.shuffle(shuffle_size, seed=777)

        test_size = int(test_split * len(ds))
        train_size = len(ds) - test_size

        self.train_ds = ds.take(train_size)
        self.test_ds = ds.take(test_size)

        return self.train_ds, self.test_ds


    def model_compile(self, learning_rate=LEARNING_RATE):
        self.model = MyModel(self.model_name)
        # 컴파일
        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE)
        self.loss = tf.keras.losses.BinaryCrossentropy()

        self.model.compile(optimizer=self.optimizer,
                           loss=self.loss,
                           metrics = ['binary_accuracy']
                           )



    def run_model(self, filepath, patience=3, batch_size=32, epochs=5):

        # 콜백함수
        # chk = tf.keras.callbacks.ModelCheckpoint(filepath=filepath+'_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss') # SubClass에서는 체크포인트 저장 불가
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=patience,
                                              restore_best_weights=True,
                                              verbose=1,
                                              start_from_epoch=4) # epoch 4부터 모니터링 시작

        # 학습
        self.history = self.model.fit(
                        self.train_ds.batch(batch_size),
                        validation_data = [self.test_ds.batch(batch_size)],
                        epochs=epochs,
                        batch_size=batch_size,
                        # callbacks=[es, chk]
                        callbacks=[es]
                        )

        self.model.save_weights(filepath=filepath + '.h5',
                                overwrite=True,
                                save_format='h5'
                                )

        return self.history

    def visualizer(self):

        fig = make_subplots(rows=1, cols=2,
                            subplot_titles=('Loss', 'Accuracy'))

        # loss
        fig.add_trace(go.Scatter(x=self.history.epoch,
                                 y=self.history.history['loss'],
                                 name='loss',
                                 mode='lines+markers'),
                      row=1, col=1)

        # val loss
        fig.add_trace(go.Scatter(x=self.history.epoch,
                                 y=self.history.history['val_loss'],
                                 name='val_loss',
                                 mode='lines+markers'),
                      row=1, col=1)

        # binary accuracy
        fig.add_trace(go.Scatter(x=self.history.epoch,
                                 y=self.history.history['binary_accuracy'],
                                 name='binary_accuracy',
                                 mode='lines+markers'),
                      row=1, col=2)

        # val binary accuracy
        fig.add_trace(go.Scatter(x=self.history.epoch,
                                 y=self.history.history['val_binary_accuracy'],
                                 name='val_binary_accuracy',
                                 mode='lines+markers'),
                      row=1, col=2)


        fig.update_layout(showlegend=True)

        fig.show()

## 모델 불러오기

In [ ]:
wf = WorkFlow(MODEL_NAME)

wf.config

BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-small-uncased",
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
# 데이터셋 지정
X_data = df['facts']
y_data = df['first_party_winner'].astype(np.int32)

train_ds, test_ds = wf.get_dataset(X_data, y_data)

In [ ]:
save_name = MODEL_NAME.split('/')[1]

# compile 및 fit
wf.model_compile()
history = wf.run_model(filepath= save_name, epochs=10)

Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 0.6659 - binary_accuracy: 0.6455

In [ ]:
import pymysql
import pandas as pd

class DbHelper():
    def __init__(self):
        pass

    def get_TrainData(self, config):
        db_connection = pymysql.connect(
           **config
        )
        cursor = db_connection.cursor()

        # Execute the SQL query to fetch the data
        sql = "SELECT * FROM devcon"
        cursor.execute(sql)

        # Fetch all the rows returned by the query
        rows = cursor.fetchall()

        data = []

        # Iterate over the rows and append data to the list as dictionaries
        for row in rows:
            raw_id = row[1]
            raw_first_party = row[2]
            raw_second_party = row[3]
            raw_facts = row[4]
            raw_winner = row[5]
            facts_split = row[6]

            # Create a dictionary for the row
            row_dict = {
                '_id': raw_id,
                'raw_first_party': raw_first_party,
                'raw_second_party': raw_second_party,
                'raw_facts': raw_facts,
                'raw_winner': raw_winner,
                'facts_split':facts_split
            }

            # Append the dictionary to the data list
            data.append(row_dict)

        # Create a DataFrame from the data list
        df = pd.DataFrame(data)
        # Close the cursor and connection
        cursor.close()
        db_connection.close()

        # Print the DataFrame
        return df

    def get_TestData(self, config):


        db_connection = pymysql.connect(
           **config
        )
        cursor = db_connection.cursor()

        # Execute the SQL query to fetch the data
        sql = "SELECT * FROM devcon_test"
        cursor.execute(sql)

        # Fetch all the rows returned by the query
        rows = cursor.fetchall()

        data = []

        # Iterate over the rows and append data to the list as dictionaries
        for row in rows:
            raw_id = row[1]
            raw_first_party = row[2]
            raw_second_party = row[3]
            raw_facts = row[4]

            # Create a dictionary for the row
            row_dict = {
                '_id': raw_id,
                'raw_first_party': raw_first_party,
                'raw_second_party': raw_second_party,
                'raw_facts': raw_facts,
            }

            # Append the dictionary to the data list
            data.append(row_dict)

        # Create a DataFrame from the data list
        df = pd.DataFrame(data)
        # Close the cursor and connection
        cursor.close()
        db_connection.close()

        # Print the DataFrame
        return df

    def update_TrainData(self, config, dataFrame):
        db_connection = pymysql.connect(
           **config
        )
        cursor = db_connection.cursor()
        sql = "update devcon set raw_facts ='%s' where raw_ID = '%s'"

        for index, row in enumerate(dataFrame):
             row['']
             cursor.execute(sql.format())

        cursor.execute(sql)
        cursor.close()
        db_connection.close()

In [ ]:
connectionInfo = {
    'host':'private.dotge.site',
    'user':'ade345',
    'password':'dbslwms123',
    'database':'books'
}
alpha = DbHelper()
test_df=alpha.get_TestData(connectionInfo)
test_df

,_id,raw_first_party,raw_second_party,raw_facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


In [ ]:
raw_facts_test  = list(map(lambda x: x.split(), test_df['raw_facts']))
raw_facts_test[:5]
raw_id_test = test_df['_id']

In [ ]:
test_x = test_df['raw_facts'].tolist()

In [ ]:
# 레이어 구조 확인
display(plot_model(wf.model, show_shapes=True))
wf.model.summary()

In [ ]:
# 시각화
wf.visualizer()

In [ ]:
# 저장된 weight 불러오기
f  = wf.model.load_weights(f"{save_name}.h5")